In [1]:
# notebook dependencies 
%matplotlib inline
import matplotlib as mlp
mlp.rcParams['figure.dpi'] = 300

import pandas as pd
import numpy as np
import os

# visualization imports
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

# time module
from time import time

from initial_acquire import get_majors_df
import mm_prepare
import mm_acquire

In [2]:
# import bach df

df = mm_acquire.get_bach_df()
df.head()

dataframe shape: (71901, 125)


,UNITID,INSTNM_x,CONTROL_x,CITY,STABBR,ZIP,PFTFTUG1_EF,PPTUG_EF,PREDDEG,REGION,...,IND_DEBT_MDN,LO_INC_DEBT_MDN,MALE_DEBT_MDN,MD_INC_DEBT_MDN,NOPELL_DEBT_MDN,NOTFIRSTGEN_DEBT_MDN,HI_INC_DEBT_MDN,GRAD_DEBT_MDN,FTFTPCTFLOAN,FTFTPCTPELL
0,100654.0,Alabama A & M University,Public,Normal,AL,35762,0.8987,0.0587,3.0,5.0,...,19000,15500,14250,16000,10250,16421,14518,33375,0.7143,0.7057
1,100654.0,Alabama A & M University,Public,Normal,AL,35762,0.8987,0.0587,3.0,5.0,...,19000,15500,14250,16000,10250,16421,14518,33375,0.7143,0.7057
2,100654.0,Alabama A & M University,Public,Normal,AL,35762,0.8987,0.0587,3.0,5.0,...,19000,15500,14250,16000,10250,16421,14518,33375,0.7143,0.7057
5,100654.0,Alabama A & M University,Public,Normal,AL,35762,0.8987,0.0587,3.0,5.0,...,19000,15500,14250,16000,10250,16421,14518,33375,0.7143,0.7057
6,100654.0,Alabama A & M University,Public,Normal,AL,35762,0.8987,0.0587,3.0,5.0,...,19000,15500,14250,16000,10250,16421,14518,33375,0.7143,0.7057


In [3]:
# clean "privacy surpressed" and col names

df = mm_prepare.clean_college_df(df)
df.head()

,unit_id_institution,college_name,institution_control,city,state_post_code,zip_code,share_entering_students_first_ft,share_of_part_time,pred_degree,religion_ipeds,...,median_debt_independent,median_debt_0_30000,median_debt_male,median_debt_30001_75000,median_debt_non_pell,median_debt_non_first_generation,median_debt_75001+,median_debt_completed,fedral_loan_full_time_first_time_undergraduate,pell_grant_full_time_first_time_undergraduate
0,100654.0,Alabama A & M University,Public,Normal,AL,35762,0.8987,0.0587,3.0,5.0,...,19000,15500,14250,16000,10250,16421,14518,33375,0.7143,0.7057
1,100654.0,Alabama A & M University,Public,Normal,AL,35762,0.8987,0.0587,3.0,5.0,...,19000,15500,14250,16000,10250,16421,14518,33375,0.7143,0.7057
2,100654.0,Alabama A & M University,Public,Normal,AL,35762,0.8987,0.0587,3.0,5.0,...,19000,15500,14250,16000,10250,16421,14518,33375,0.7143,0.7057
5,100654.0,Alabama A & M University,Public,Normal,AL,35762,0.8987,0.0587,3.0,5.0,...,19000,15500,14250,16000,10250,16421,14518,33375,0.7143,0.7057
6,100654.0,Alabama A & M University,Public,Normal,AL,35762,0.8987,0.0587,3.0,5.0,...,19000,15500,14250,16000,10250,16421,14518,33375,0.7143,0.7057


In [6]:
# check function worked?

if "PrivacySuppressed" in df.values:
    print(True)
else: print(False)

False


In [13]:
def get_majors_df():

    '''Function to initially pull and merge the two (2) needed 
    College Scorecard tables for period 2018-2019.'''

    # checking if dataset exists
    filename = "majors_table.csv"
    
    if os.path.isfile(filename):
        
        df = pd.read_csv(filename, index_col=0)

        print(f'dataframe shape: {df.shape}')

        return df

    else:
        # checks local foldere for following files
        filename_01 = "FieldOfStudyData1718_1819_PP.csv"
        filename_02 = "MERGED2018_19_PP.csv"
        
        # created the necessary parent and child tables
        df_parent = pd.read_csv(filename_01, low_memory=False)
        df_child = pd.read_csv(filename_02, low_memory=False)

        df_parent["UNITID"] = df_parent["UNITID"].astype("Int32", errors='ignore')
        df_child["UNITID"] = df_child["UNITID"].astype("Int32", errors='ignore')

        df = df_parent.merge( 
        df_child,
        how = "left",
        on = "UNITID",
        copy = False
        )
        # cache the newly created dataframe as a .csv file
        df.to_csv("majors_table.csv")
        # print the df shape
        print(f'dataframe shape: {df.shape}')

        # return the dataframe
        return df

In [14]:
def get_bach_df():

    '''Function to initially pull and merge the two (2) needed 
    College Scorecard tables for period 2018-2019.
    
    The function then filters and returns bachelor degree records.'''

    # checking if dataset exists
    filename = "bach_table.csv"
    
    if os.path.isfile(filename):
        
        df = pd.read_csv(filename, index_col=0)

        print(f'dataframe shape: {df.shape}')

        return df

    else:
        # checks local foldere for following files
        filename_01 = "FieldOfStudyData1718_1819_PP.csv"
        filename_02 = "MERGED2018_19_PP.csv"
        
        # created the necessary parent and child tables
        df_parent = pd.read_csv(filename_01, low_memory=False)
        df_child = pd.read_csv(filename_02, low_memory=False)

        df_parent["UNITID"] = df_parent["UNITID"].astype("Int32", errors='ignore')
        df_child["UNITID"] = df_child["UNITID"].astype("Int32", errors='ignore')

        df = df_parent.merge( 
        df_child,
        how = "left",
        on = "UNITID",
        copy = False
        )

        # filters for just bachelor specific records
        bach_df = df[df["CREDDESC"] == "Bachelors Degree"]

        # initial filter of columns with >= 50% missing records
        bach_df = bach_df[[ 
                "INSTNM_x",
                "CONTROL_x",
                "CITY",
                "STABBR",
                "ZIP",
                "PFTFTUG1_EF",
                "PPTUG_EF",
                "PREDDEG",
                "REGION",
                "RET_FT4",
                "RET_PT4",
                "ROOMBOARD_OFF",
                "ROOMBOARD_ON",
                "SAT_AVG",
                "SCH_DEG",
                "TUITFTE",
                "ACTCMMID",
                "ADM_RATE",
                "ADMCON7",
                "AVGFACSAL",
                "CREDDESC",
                "CREDLEV",
                "CIPCODE",
                "CIPDESC",
                "DISTANCEONLY",
                "GRADS",
                "NUM4_PRIV",
                "NUMBRANCH",
                "NPT4_PRIV",
                "NPT4_PUB",
                "OPEFLAG",
                "NUM41_OTHER",
                "NUM41_PRIV",
                "NUM41_PROG",
                "NUM41_PUB",
                "NUM42_OTHER",
                "NUM42_PRIV",
                "NUM42_PROG",
                "NUM42_PUB",
                "NUM43_OTHER",
                "NUM43_PRIV",
                "NUM43_PROG",
                "NUM43_PUB",
                "NUM44_OTHER",
                "NUM44_PRIV",
                "NUM44_PROG",
                "NUM44_PUB",
                "NUM45_OTHER",
                "NUM45_PRIV",
                "NUM45_PROG",
                "NUM45_PUB",
                "PCIP01",
                "PCIP03",
                "PCIP04",
                "PCIP05",
                "PCIP09",
                "PCIP10",
                "PCIP11",
                "PCIP12",
                "PCIP13",
                "PCIP14",
                "PCIP15",
                "PCIP16",
                "PCIP19",
                "PCIP22",
                "PCIP23",
                "PCIP24",
                "PCIP25",
                "PCIP26",
                "PCIP27",
                "PCIP29",
                "PCIP30",
                "PCIP31",
                "PCIP38",
                "PCIP39",
                "PCIP40",
                "PCIP41",
                "PCIP42",
                "PCIP43",
                "PCIP44",
                "PCIP45",
                "PCIP46",
                "PCIP47",
                "PCIP48",
                "PCIP49",
                "PCIP50",
                "PCIP51",
                "PCIP52",
                "PCIP54",
                "C150_4_2MOR",
                "C150_4_AIAN",
                "C150_4_ASIAN",
                "C150_4_BLACK",
                "C150_4_HISP",
                "C150_4_NRA",
                "C150_4_UNKN",
                "C150_4_WHITE",
                "C150_4",
                "UGDS_2MOR",
                "UGDS_AIAN",
                "UGDS_ASIAN",
                "UGDS_BLACK",
                "UGDS_HISP",
                "UGDS_NHPI",
                "UGDS_NRA",
                "UGDS_UNKN",
                "UGDS_WHITE",
                "UGNONDS",
                "UNITID",
                "D_PCTPELL_PCTFLOAN",
                "DEBT_MDN",
                "PELL_DEBT_MDN",
                "WDRAW_DEBT_MDN",
                "FEMALE_DEBT_MDN",
                "FIRSTGEN_DEBT_MDN",
                "IND_DEBT_MDN",
                "LO_INC_DEBT_MDN",
                "MALE_DEBT_MDN",
                "MD_INC_DEBT_MDN",
                "NOPELL_DEBT_MDN",
                "NOTFIRSTGEN_DEBT_MDN",
                "HI_INC_DEBT_MDN",
                "GRAD_DEBT_MDN",
                "FTFTPCTFLOAN",
                "FTFTPCTPELL"]]
            
        # cache the newly created dataframe as a .csv file
        bach_df.to_csv("bach_table.csv")
        
        # print the df shape
        print(f"dataframe shape: {bach_df.shape}")

        # return the dataframe
        return bach_df

In [3]:
# test the function

df = mm_acquire.get_bach_df()
df.head()

dataframe shape: (71901, 125)


,UNITID,INSTNM_x,CONTROL_x,CITY,STABBR,ZIP,PFTFTUG1_EF,PPTUG_EF,PREDDEG,REGION,...,IND_DEBT_MDN,LO_INC_DEBT_MDN,MALE_DEBT_MDN,MD_INC_DEBT_MDN,NOPELL_DEBT_MDN,NOTFIRSTGEN_DEBT_MDN,HI_INC_DEBT_MDN,GRAD_DEBT_MDN,FTFTPCTFLOAN,FTFTPCTPELL
0,100654.0,Alabama A & M University,Public,Normal,AL,35762,0.8987,0.0587,3.0,5.0,...,19000,15500,14250,16000,10250,16421,14518,33375,0.7143,0.7057
1,100654.0,Alabama A & M University,Public,Normal,AL,35762,0.8987,0.0587,3.0,5.0,...,19000,15500,14250,16000,10250,16421,14518,33375,0.7143,0.7057
2,100654.0,Alabama A & M University,Public,Normal,AL,35762,0.8987,0.0587,3.0,5.0,...,19000,15500,14250,16000,10250,16421,14518,33375,0.7143,0.7057
5,100654.0,Alabama A & M University,Public,Normal,AL,35762,0.8987,0.0587,3.0,5.0,...,19000,15500,14250,16000,10250,16421,14518,33375,0.7143,0.7057
6,100654.0,Alabama A & M University,Public,Normal,AL,35762,0.8987,0.0587,3.0,5.0,...,19000,15500,14250,16000,10250,16421,14518,33375,0.7143,0.7057


In [4]:
df = mm_prepare.clean_col_names(df)
# df = mm_prepare.clean_bach_df(df)
df.head()

,unit_id_institution,college_name,institution_control,city,state_post_code,zip_code,share_entering_students_first_ft,share_of_part_time,pred_degree,religion_ipeds,...,median_debt_independent,median_debt_0_30000,median_debt_male,median_debt_30001_75000,median_debt_non_pell,median_debt_non_first_generation,median_debt_75001+,median_debt_completed,fedral_loan_full_time_first_time_undergraduate,pell_grant_full_time_first_time_undergraduate
0,100654.0,Alabama A & M University,Public,Normal,AL,35762,0.8987,0.0587,3.0,5.0,...,19000,15500,14250,16000,10250,16421,14518,33375,0.7143,0.7057
1,100654.0,Alabama A & M University,Public,Normal,AL,35762,0.8987,0.0587,3.0,5.0,...,19000,15500,14250,16000,10250,16421,14518,33375,0.7143,0.7057
2,100654.0,Alabama A & M University,Public,Normal,AL,35762,0.8987,0.0587,3.0,5.0,...,19000,15500,14250,16000,10250,16421,14518,33375,0.7143,0.7057
5,100654.0,Alabama A & M University,Public,Normal,AL,35762,0.8987,0.0587,3.0,5.0,...,19000,15500,14250,16000,10250,16421,14518,33375,0.7143,0.7057
6,100654.0,Alabama A & M University,Public,Normal,AL,35762,0.8987,0.0587,3.0,5.0,...,19000,15500,14250,16000,10250,16421,14518,33375,0.7143,0.7057


In [4]:
# create a masters table

def get_masters_df():

    '''Function to initially pull and merge the two (2) needed 
    College Scorecard tables for period 2018-2019.
    
    The function then filters and returns Masters degree records.'''

    # checking if dataset exists
    filename = "masters_table.csv"
    
    if os.path.isfile(filename):
        
        df = pd.read_csv(filename, index_col=0)

        print(f'dataframe shape: {df.shape}')

        return df

    else:
        # checks local foldere for following files
        filename_01 = "FieldOfStudyData1718_1819_PP.csv"
        filename_02 = "MERGED2018_19_PP.csv"
        
        # created the necessary parent and child tables
        df_parent = pd.read_csv(filename_01, low_memory=False)
        df_child = pd.read_csv(filename_02, low_memory=False)

        df_parent["UNITID"] = df_parent["UNITID"].astype("Int32", errors='ignore')
        df_child["UNITID"] = df_child["UNITID"].astype("Int32", errors='ignore')

        df = df_parent.merge( 
        df_child,
        how = "left",
        on = "UNITID",
        copy = False
        )

        # filters for just bachelor specific records
        masters_df = df[df["CREDDESC"] == "Master's Degree"]

        # initial filter of columns with >= 50% missing records
        masters_df = masters_df[[ 
                "UNITID",
                "INSTNM_x",
                "CONTROL_x",
                "CITY",
                "STABBR",
                "ZIP",
                "PFTFTUG1_EF",
                "PPTUG_EF",
                "PREDDEG",
                "REGION",
                "RET_FT4",
                "RET_PT4",
                "ROOMBOARD_OFF",
                "ROOMBOARD_ON",
                "SAT_AVG",
                "SCH_DEG",
                "TUITFTE",
                "ACTCMMID",
                "ADM_RATE",
                "ADMCON7",
                "AVGFACSAL",
                "CREDDESC",
                "CREDLEV",
                "CIPCODE",
                "CIPDESC",
                "DISTANCEONLY",
                "GRADS",
                "NUM4_PRIV",
                "NUMBRANCH",
                "NPT4_PRIV",
                "NPT4_PUB",
                "OPEFLAG",
                "NUM41_OTHER",
                "NUM41_PRIV",
                "NUM41_PROG",
                "NUM41_PUB",
                "NUM42_OTHER",
                "NUM42_PRIV",
                "NUM42_PROG",
                "NUM42_PUB",
                "NUM43_OTHER",
                "NUM43_PRIV",
                "NUM43_PROG",
                "NUM43_PUB",
                "NUM44_OTHER",
                "NUM44_PRIV",
                "NUM44_PROG",
                "NUM44_PUB",
                "NUM45_OTHER",
                "NUM45_PRIV",
                "NUM45_PROG",
                "NUM45_PUB",
                "PCIP01",
                "PCIP03",
                "PCIP04",
                "PCIP05",
                "PCIP09",
                "PCIP10",
                "PCIP11",
                "PCIP12",
                "PCIP13",
                "PCIP14",
                "PCIP15",
                "PCIP16",
                "PCIP19",
                "PCIP22",
                "PCIP23",
                "PCIP24",
                "PCIP25",
                "PCIP26",
                "PCIP27",
                "PCIP29",
                "PCIP30",
                "PCIP31",
                "PCIP38",
                "PCIP39",
                "PCIP40",
                "PCIP41",
                "PCIP42",
                "PCIP43",
                "PCIP44",
                "PCIP45",
                "PCIP46",
                "PCIP47",
                "PCIP48",
                "PCIP49",
                "PCIP50",
                "PCIP51",
                "PCIP52",
                "PCIP54",
                "C150_4_2MOR",
                "C150_4_AIAN",
                "C150_4_ASIAN",
                "C150_4_BLACK",
                "C150_4_HISP",
                "C150_4_NRA",
                "C150_4_UNKN",
                "C150_4_WHITE",
                "C150_4",
                "UGDS_2MOR",
                "UGDS_AIAN",
                "UGDS_ASIAN",
                "UGDS_BLACK",
                "UGDS_HISP",
                "UGDS_NHPI",
                "UGDS_NRA",
                "UGDS_UNKN",
                "UGDS_WHITE",
                "UGNONDS",
                "D_PCTPELL_PCTFLOAN",
                "DEBT_MDN",
                "PELL_DEBT_MDN",
                "WDRAW_DEBT_MDN",
                "FEMALE_DEBT_MDN",
                "FIRSTGEN_DEBT_MDN",
                "IND_DEBT_MDN",
                "LO_INC_DEBT_MDN",
                "MALE_DEBT_MDN",
                "MD_INC_DEBT_MDN",
                "NOPELL_DEBT_MDN",
                "NOTFIRSTGEN_DEBT_MDN",
                "HI_INC_DEBT_MDN",
                "GRAD_DEBT_MDN",
                "FTFTPCTFLOAN",
                "FTFTPCTPELL"]]
            
        # cache the newly created dataframe as a .csv file
        masters_df.to_csv("masters_table.csv")
        
        # print the df shape
        print(f'dataframe shape: {masters_df.shape}')

        # return the dataframe
        return masters_df

In [6]:
# completing the cache function

df = get_masters_df()

dataframe shape: (37939, 126)


In [3]:
# creating a phD function 

def get_phd_df():

    '''Function to initially pull and merge the two (2) needed 
    College Scorecard tables for period 2018-2019.
    
    The function then filters and returns doctoral degree records.'''

    # checking if dataset exists
    filename = "phd_table.csv"
    
    if os.path.isfile(filename):
        
        df = pd.read_csv(filename, index_col=0)

        print(f'dataframe shape: {df.shape}')

        return df

    else:
        # checks local foldere for following files
        filename_01 = "FieldOfStudyData1718_1819_PP.csv"
        filename_02 = "MERGED2018_19_PP.csv"
        
        # created the necessary parent and child tables
        df_parent = pd.read_csv(filename_01, low_memory=False)
        df_child = pd.read_csv(filename_02, low_memory=False)

        df_parent["UNITID"] = df_parent["UNITID"].astype("Int32", errors='ignore')
        df_child["UNITID"] = df_child["UNITID"].astype("Int32", errors='ignore')

        df = df_parent.merge( 
        df_child,
        how = "left",
        on = "UNITID",
        copy = False
        )

        # filters for just bachelor specific records
        phd_df = df[df["CREDDESC"] == "Doctoral Degree"]

        # initial filter of columns with >= 50% missing records
        phd_df = phd_df[[ 
                "UNITID",
                "INSTNM_x",
                "CONTROL_x",
                "CITY",
                "STABBR",
                "ZIP",
                "PFTFTUG1_EF",
                "PPTUG_EF",
                "PREDDEG",
                "REGION",
                "RET_FT4",
                "RET_PT4",
                "ROOMBOARD_OFF",
                "ROOMBOARD_ON",
                "SAT_AVG",
                "SCH_DEG",
                "TUITFTE",
                "ACTCMMID",
                "ADM_RATE",
                "ADMCON7",
                "AVGFACSAL",
                "CREDDESC",
                "CREDLEV",
                "CIPCODE",
                "CIPDESC",
                "DISTANCEONLY",
                "GRADS",
                "NUM4_PRIV",
                "NUMBRANCH",
                "NPT4_PRIV",
                "NPT4_PUB",
                "OPEFLAG",
                "NUM41_OTHER",
                "NUM41_PRIV",
                "NUM41_PROG",
                "NUM41_PUB",
                "NUM42_OTHER",
                "NUM42_PRIV",
                "NUM42_PROG",
                "NUM42_PUB",
                "NUM43_OTHER",
                "NUM43_PRIV",
                "NUM43_PROG",
                "NUM43_PUB",
                "NUM44_OTHER",
                "NUM44_PRIV",
                "NUM44_PROG",
                "NUM44_PUB",
                "NUM45_OTHER",
                "NUM45_PRIV",
                "NUM45_PROG",
                "NUM45_PUB",
                "PCIP01",
                "PCIP03",
                "PCIP04",
                "PCIP05",
                "PCIP09",
                "PCIP10",
                "PCIP11",
                "PCIP12",
                "PCIP13",
                "PCIP14",
                "PCIP15",
                "PCIP16",
                "PCIP19",
                "PCIP22",
                "PCIP23",
                "PCIP24",
                "PCIP25",
                "PCIP26",
                "PCIP27",
                "PCIP29",
                "PCIP30",
                "PCIP31",
                "PCIP38",
                "PCIP39",
                "PCIP40",
                "PCIP41",
                "PCIP42",
                "PCIP43",
                "PCIP44",
                "PCIP45",
                "PCIP46",
                "PCIP47",
                "PCIP48",
                "PCIP49",
                "PCIP50",
                "PCIP51",
                "PCIP52",
                "PCIP54",
                "C150_4_2MOR",
                "C150_4_AIAN",
                "C150_4_ASIAN",
                "C150_4_BLACK",
                "C150_4_HISP",
                "C150_4_NRA",
                "C150_4_UNKN",
                "C150_4_WHITE",
                "C150_4",
                "UGDS_2MOR",
                "UGDS_AIAN",
                "UGDS_ASIAN",
                "UGDS_BLACK",
                "UGDS_HISP",
                "UGDS_NHPI",
                "UGDS_NRA",
                "UGDS_UNKN",
                "UGDS_WHITE",
                "UGNONDS",
                "D_PCTPELL_PCTFLOAN",
                "DEBT_MDN",
                "PELL_DEBT_MDN",
                "WDRAW_DEBT_MDN",
                "FEMALE_DEBT_MDN",
                "FIRSTGEN_DEBT_MDN",
                "IND_DEBT_MDN",
                "LO_INC_DEBT_MDN",
                "MALE_DEBT_MDN",
                "MD_INC_DEBT_MDN",
                "NOPELL_DEBT_MDN",
                "NOTFIRSTGEN_DEBT_MDN",
                "HI_INC_DEBT_MDN",
                "GRAD_DEBT_MDN",
                "FTFTPCTFLOAN",
                "FTFTPCTPELL"]]
            
        # cache the newly created dataframe as a .csv file
        phd_df.to_csv("phd_df.csv")
        
        # print the df shape
        print(f'dataframe shape: {phd_df.shape}')

        # return the dataframe
        return phd_df

In [7]:
# running the function

df = get_phd_df()
df.head()

dataframe shape: (12494, 125)


,UNITID,INSTNM_x,CONTROL_x,CITY,STABBR,ZIP,PFTFTUG1_EF,PPTUG_EF,PREDDEG,REGION,...,IND_DEBT_MDN,LO_INC_DEBT_MDN,MALE_DEBT_MDN,MD_INC_DEBT_MDN,NOPELL_DEBT_MDN,NOTFIRSTGEN_DEBT_MDN,HI_INC_DEBT_MDN,GRAD_DEBT_MDN,FTFTPCTFLOAN,FTFTPCTPELL
4,100654,Alabama A & M University,Public,Normal,AL,35762,0.8987,0.0587,3.0,5.0,...,19000,15500,14250,16000,10250,16421,14518,33375,0.7143,0.7057
8,100654,Alabama A & M University,Public,Normal,AL,35762,0.8987,0.0587,3.0,5.0,...,19000,15500,14250,16000,10250,16421,14518,33375,0.7143,0.7057
25,100654,Alabama A & M University,Public,Normal,AL,35762,0.8987,0.0587,3.0,5.0,...,19000,15500,14250,16000,10250,16421,14518,33375,0.7143,0.7057
66,100663,University of Alabama at Birmingham,Public,Birmingham,AL,35294-0110,0.6009,0.2425,3.0,5.0,...,16000,16226,15000,15250,12500,15000,14000,22500,0.5350,0.3788
73,100663,University of Alabama at Birmingham,Public,Birmingham,AL,35294-0110,0.6009,0.2425,3.0,5.0,...,16000,16226,15000,15250,12500,15000,14000,22500,0.5350,0.3788


In [6]:
# caching associate degree tables

# creating a phD function 

def get_associates_df():

    '''Function to initially pull and merge the two (2) needed 
    College Scorecard tables for period 2018-2019.
    
    The function then filters and returns associate degree records.'''

    # checking if dataset exists
    filename = "associates_table.csv"
    
    if os.path.isfile(filename):
        
        df = pd.read_csv(filename, index_col=0)

        print(f'dataframe shape: {df.shape}')

        return df

    else:
        # checks local foldere for following files
        filename_01 = "FieldOfStudyData1718_1819_PP.csv"
        filename_02 = "MERGED2018_19_PP.csv"
        
        # created the necessary parent and child tables
        df_parent = pd.read_csv(filename_01, low_memory=False)
        df_child = pd.read_csv(filename_02, low_memory=False)

        df_parent["UNITID"] = df_parent["UNITID"].astype("Int32", errors='ignore')
        df_child["UNITID"] = df_child["UNITID"].astype("Int32", errors='ignore')

        df = df_parent.merge( 
        df_child,
        how = "left",
        on = "UNITID",
        copy = False
        )

        # filters for just bachelor specific records
        assoc_df = df[df["CREDDESC"] == "Associate's Degree"]

        # initial filter of columns with >= 50% missing records
        assoc_df = assoc_df[[ 
                "UNITID",
                "INSTNM_x",
                "CONTROL_x",
                "CITY",
                "STABBR",
                "ZIP",
                "PFTFTUG1_EF",
                "PPTUG_EF",
                "PREDDEG",
                "REGION",
                "RET_FT4",
                "RET_PT4",
                "ROOMBOARD_OFF",
                "ROOMBOARD_ON",
                "SAT_AVG",
                "SCH_DEG",
                "TUITFTE",
                "ACTCMMID",
                "ADM_RATE",
                "ADMCON7",
                "AVGFACSAL",
                "CREDDESC",
                "CREDLEV",
                "CIPCODE",
                "CIPDESC",
                "DISTANCEONLY",
                "GRADS",
                "NUM4_PRIV",
                "NUMBRANCH",
                "NPT4_PRIV",
                "NPT4_PUB",
                "OPEFLAG",
                "NUM41_OTHER",
                "NUM41_PRIV",
                "NUM41_PROG",
                "NUM41_PUB",
                "NUM42_OTHER",
                "NUM42_PRIV",
                "NUM42_PROG",
                "NUM42_PUB",
                "NUM43_OTHER",
                "NUM43_PRIV",
                "NUM43_PROG",
                "NUM43_PUB",
                "NUM44_OTHER",
                "NUM44_PRIV",
                "NUM44_PROG",
                "NUM44_PUB",
                "NUM45_OTHER",
                "NUM45_PRIV",
                "NUM45_PROG",
                "NUM45_PUB",
                "PCIP01",
                "PCIP03",
                "PCIP04",
                "PCIP05",
                "PCIP09",
                "PCIP10",
                "PCIP11",
                "PCIP12",
                "PCIP13",
                "PCIP14",
                "PCIP15",
                "PCIP16",
                "PCIP19",
                "PCIP22",
                "PCIP23",
                "PCIP24",
                "PCIP25",
                "PCIP26",
                "PCIP27",
                "PCIP29",
                "PCIP30",
                "PCIP31",
                "PCIP38",
                "PCIP39",
                "PCIP40",
                "PCIP41",
                "PCIP42",
                "PCIP43",
                "PCIP44",
                "PCIP45",
                "PCIP46",
                "PCIP47",
                "PCIP48",
                "PCIP49",
                "PCIP50",
                "PCIP51",
                "PCIP52",
                "PCIP54",
                "C150_4_2MOR",
                "C150_4_AIAN",
                "C150_4_ASIAN",
                "C150_4_BLACK",
                "C150_4_HISP",
                "C150_4_NRA",
                "C150_4_UNKN",
                "C150_4_WHITE",
                "C150_4",
                "UGDS_2MOR",
                "UGDS_AIAN",
                "UGDS_ASIAN",
                "UGDS_BLACK",
                "UGDS_HISP",
                "UGDS_NHPI",
                "UGDS_NRA",
                "UGDS_UNKN",
                "UGDS_WHITE",
                "UGNONDS",
                "D_PCTPELL_PCTFLOAN",
                "DEBT_MDN",
                "PELL_DEBT_MDN",
                "WDRAW_DEBT_MDN",
                "FEMALE_DEBT_MDN",
                "FIRSTGEN_DEBT_MDN",
                "IND_DEBT_MDN",
                "LO_INC_DEBT_MDN",
                "MALE_DEBT_MDN",
                "MD_INC_DEBT_MDN",
                "NOPELL_DEBT_MDN",
                "NOTFIRSTGEN_DEBT_MDN",
                "HI_INC_DEBT_MDN",
                "GRAD_DEBT_MDN",
                "FTFTPCTFLOAN",
                "FTFTPCTPELL"]]
            
        # cache the newly created dataframe as a .csv file
        assoc_df.to_csv("associates_table.csv")
        
        # print the df shape
        print(f'dataframe shape: {assoc_df.shape}')

        # return the dataframe
        return assoc_df
    

In [7]:
# running the function

df = get_associates_df()
df.head()

dataframe shape: (43706, 125)


,UNITID,INSTNM_x,CONTROL_x,CITY,STABBR,ZIP,PFTFTUG1_EF,PPTUG_EF,PREDDEG,REGION,...,IND_DEBT_MDN,LO_INC_DEBT_MDN,MALE_DEBT_MDN,MD_INC_DEBT_MDN,NOPELL_DEBT_MDN,NOTFIRSTGEN_DEBT_MDN,HI_INC_DEBT_MDN,GRAD_DEBT_MDN,FTFTPCTFLOAN,FTFTPCTPELL
220,100690,Amridge University,"Private, nonprofit",Montgomery,AL,36117-3553,0.0035,0.5014,3.0,5.0,...,12038,9500,17025,19000,15509,9500,25000,27334,1.0,1.0
223,100690,Amridge University,"Private, nonprofit",Montgomery,AL,36117-3553,0.0035,0.5014,3.0,5.0,...,12038,9500,17025,19000,15509,9500,25000,27334,1.0,1.0
225,100690,Amridge University,"Private, nonprofit",Montgomery,AL,36117-3553,0.0035,0.5014,3.0,5.0,...,12038,9500,17025,19000,15509,9500,25000,27334,1.0,1.0
234,100690,Amridge University,"Private, nonprofit",Montgomery,AL,36117-3553,0.0035,0.5014,3.0,5.0,...,12038,9500,17025,19000,15509,9500,25000,27334,1.0,1.0
238,100690,Amridge University,"Private, nonprofit",Montgomery,AL,36117-3553,0.0035,0.5014,3.0,5.0,...,12038,9500,17025,19000,15509,9500,25000,27334,1.0,1.0


In [28]:
# identifying unique college establishments 

cols_lst = [
"HBCU",
"PBI",
"ANNHI",
"TRIBAL",
"AANAPII",
"HSI",
"NANTI",
"MENONLY",
"WOMENONLY",
"RELAFFIL"]

for column in cols_lst:
    print(f'col name: {str(df[column].name)}')
    print(f'unique name: {df[column].unique()}')

col name: HBCU
unique name: [nan]
col name: PBI
unique name: [nan]
col name: ANNHI
unique name: [nan]
col name: TRIBAL
unique name: [nan]
col name: AANAPII
unique name: [nan]
col name: HSI
unique name: [nan]
col name: NANTI
unique name: [nan]
col name: MENONLY
unique name: [nan]
col name: WOMENONLY
unique name: [nan]
col name: RELAFFIL
unique name: [nan]


In [29]:
# costs variables

col_lst = [
"MDCOMP_PD",
"MDCOST_PD",
"MDEARN_PD",
"MDCOMP_ALL",
"MDCOST_ALL",
"MDEARN_ALL"]

for col in col_lst:
    print(f'col name: {str(df[col].name)}')
    print(f'unique name: {df[col].unique()}')

col name: MDCOMP_PD
unique name: [nan]
col name: MDCOST_PD
unique name: [nan]
col name: MDEARN_PD
unique name: [nan]
col name: MDCOMP_ALL
unique name: [nan]
col name: MDCOST_ALL
unique name: [nan]
col name: MDEARN_ALL
unique name: [nan]


In [30]:
# not completely missing
# "NPT4_PRIV"
# "NPT4_PUB"

col_lst = ["NPT4_PUB",
           "NPT4_PRIV",
            "NPT4_PROG",
            "NPT4_OTHER"]

for col in col_lst:
    print(f'col name: {str(df[col].name)}')
    print(f'missing % value: {df[col].isnull().mean()}')

col name: NPT4_PUB
missing % value: 0.431449550587283
col name: NPT4_PRIV
missing % value: 0.6566095468514426
col name: NPT4_PROG
missing % value: 1.0
col name: NPT4_OTHER
missing % value: 1.0


In [31]:
# exploring Average net price for Title IV institutions (public institutions)
# are all public institutions == "NPT4_PUB" ???
# no, there are two (2) records not classied as public

# pd.crosstab(df["NPT4_PUB"], df["CONTROL_x"], normalize = False).sort_values(by="Private, for-profit", ascending = False)

# percentage?
pd.crosstab(df["NPT4_PUB"], df["CONTROL_x"], normalize = True).sort_values(by="Private, for-profit", ascending = False)

CONTROL_x,"Private, for-profit",Public
NPT4_PUB,,
10592.0,0.000016,0.000000
-1973.0,0.000000,0.000102
11644.0,0.000000,0.003246
11984.0,0.000000,0.000094
11948.0,0.000000,0.002175
...,...,...
7054.0,0.000000,0.000344
7052.0,0.000000,0.000399
7029.0,0.000000,0.000516


In [32]:
# exploring Average net price for Title IV institutions (private for-profit and nonprofit institutions)
# are all public institutions == "NPT4_PRIV" ???

# pd.crosstab(df["NPT4_PRIV"], df["CONTROL_x"], normalize = False).sort_values(by="Public", ascending = False)

# ok, what total of public institutions makeup this category?
pd.crosstab(df["NPT4_PRIV"], df["CONTROL_x"], normalize = True).sort_values(by="Public", ascending = False)

CONTROL_x,"Private, for-profit","Private, nonprofit",Public
NPT4_PRIV,,,
11906.0,0.000000,0.000000,0.000881
7274.0,0.000000,0.000000,0.000531
27000.0,0.000065,0.000000,0.000233
22107.0,0.000000,0.000661,0.000000
21985.0,0.000013,0.000609,0.000000
...,...,...,...
15411.0,0.000013,0.000000,0.000000
15413.0,0.000000,0.000570,0.000000
15414.0,0.000000,0.000622,0.000000


In [33]:
df[(df["NPT4_PRIV"] == 11906.0) & (df["CONTROL_x"] == "Public")]

,Unnamed: 0,UNITID,OPEID6_x,INSTNM_x,CONTROL_x,MAIN_x,CIPCODE,CIPDESC,CREDLEV,CREDDESC,...,COUNT_WNE_MALE1_P8,MD_EARN_WNE_MALE1_P8,GT_THRESHOLD_P10,MD_EARN_WNE_INC1_P10,MD_EARN_WNE_INC2_P10,MD_EARN_WNE_INC3_P10,MD_EARN_WNE_INDEP1_P10,MD_EARN_WNE_INDEP0_P10,MD_EARN_WNE_MALE0_P10,MD_EARN_WNE_MALE1_P10
157788,157788,219374.0,21437,Sinte Gleska University,Public,1,301,Natural Resources Conservation and Research.,1,Undergraduate Certificate or Diploma,...,105.0,18115.0,0.3122,17507.0,33866.0,NaN,17831.0,22354.0,21650.0,16110.0
157789,157789,219374.0,21437,Sinte Gleska University,Public,1,301,Natural Resources Conservation and Research.,2,Associate's Degree,...,105.0,18115.0,0.3122,17507.0,33866.0,NaN,17831.0,22354.0,21650.0,16110.0
157790,157790,219374.0,21437,Sinte Gleska University,Public,1,301,Natural Resources Conservation and Research.,3,Bachelors Degree,...,105.0,18115.0,0.3122,17507.0,33866.0,NaN,17831.0,22354.0,21650.0,16110.0
157791,157791,219374.0,21437,Sinte Gleska University,Public,1,399,"Natural Resources and Conservation, Other.",2,Associate's Degree,...,105.0,18115.0,0.3122,17507.0,33866.0,NaN,17831.0,22354.0,21650.0,16110.0
157792,157792,219374.0,21437,Sinte Gleska University,Public,1,502,"Ethnic, Cultural Minority, Gender, and Group S...",2,Associate's Degree,...,105.0,18115.0,0.3122,17507.0,33866.0,NaN,17831.0,22354.0,21650.0,16110.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157851,157851,219374.0,21437,Sinte Gleska University,Public,1,5203,Accounting and Related Services.,2,Associate's Degree,...,105.0,18115.0,0.3122,17507.0,33866.0,NaN,17831.0,22354.0,21650.0,16110.0
157852,157852,219374.0,21437,Sinte Gleska University,Public,1,5203,Accounting and Related Services.,3,Bachelors Degree,...,105.0,18115.0,0.3122,17507.0,33866.0,NaN,17831.0,22354.0,21650.0,16110.0
157853,157853,219374.0,21437,Sinte Gleska University,Public,1,5204,Business Operations Support and Assistant Serv...,1,Undergraduate Certificate or Diploma,...,105.0,18115.0,0.3122,17507.0,33866.0,NaN,17831.0,22354.0,21650.0,16110.0
157854,157854,219374.0,21437,Sinte Gleska University,Public,1,5204,Business Operations Support and Assistant Serv...,2,Associate's Degree,...,105.0,18115.0,0.3122,17507.0,33866.0,NaN,17831.0,22354.0,21650.0,16110.0


In [34]:
check_lst = [
"NPT4_PRIV",
"NPT4_PUB",
"CONTROL_x",
'NUM41_PUB',
'NUM41_PRIV',
'NUM41_PROG',
'NUM41_OTHER',
'NUM42_PUB',
'NUM42_PRIV',
'NUM42_PROG',
'NUM42_OTHER',
'NUM43_PUB',
'NUM43_PRIV',
'NUM43_PROG',
'NUM43_OTHER',
'NUM44_PUB',
'NUM44_PRIV',
'NUM44_PROG',
'NUM44_OTHER',
'NUM45_PUB',
'NUM45_PRIV',
'NUM45_PROG',
'NUM45_OTHER']

bach_df = get_bach_df()

for col in check_lst:
    if col in bach_df.columns:
        print(True)
    else: print(False)

dataframe shape: (71901, 134)
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [35]:
bach_df

,ACTCMMID,ADM_RATE,ADMCON7,AVGFACSAL,C150_4_2MOR,C150_4_AIAN,C150_4_ASIAN,C150_4_BLACK,C150_4_HISP,C150_4_NRA,...,NUM43_PROG,NUM43_OTHER,NUM44_PUB,NUM44_PRIV,NUM44_PROG,NUM44_OTHER,NUM45_PUB,NUM45_PRIV,NUM45_PROG,NUM45_OTHER
0,18.0,0.8986,1.0,7101.0,0.25,NaN,NaN,0.2681,0.25,NaN,...,NaN,NaN,27.0,NaN,NaN,NaN,16.0,NaN,NaN,NaN
1,18.0,0.8986,1.0,7101.0,0.25,NaN,NaN,0.2681,0.25,NaN,...,NaN,NaN,27.0,NaN,NaN,NaN,16.0,NaN,NaN,NaN
2,18.0,0.8986,1.0,7101.0,0.25,NaN,NaN,0.2681,0.25,NaN,...,NaN,NaN,27.0,NaN,NaN,NaN,16.0,NaN,NaN,NaN
5,18.0,0.8986,1.0,7101.0,0.25,NaN,NaN,0.2681,0.25,NaN,...,NaN,NaN,27.0,NaN,NaN,NaN,16.0,NaN,NaN,NaN
6,18.0,0.8986,1.0,7101.0,0.25,NaN,NaN,0.2681,0.25,NaN,...,NaN,NaN,27.0,NaN,NaN,NaN,16.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224794,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
224797,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
224812,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
224813,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
